In [9]:
print('pip install HSE')
# !pip install pandas
# !pip install pyarrow
# !pip install tqdm

pip install HSE


In [5]:
import pandas as pd
import pyarrow.parquet as pq
import datetime
import time
import tqdm
from threading import Thread
import re
import os

def parquet_to_csv(paths): # "./data/RowData/player_log/player=1547/month=2020-11.parquet"
    for path in paths:
        folder = re.sub(os.path.basename(path), '', path)
        filename = re.sub('parquet', 'csv', os.path.basename(path))
        df = pq.read_table(source=path).to_pandas()
        file = open(f"{folder}/{filename}", 'w')
        file.write('month,week_day,half_hour,efficiency,unix,views\n')
        for i in tqdm.tqdm(range(max(df.shape[0], 10))):
#         for i in range(max(df.shape[0], 10)):
            obj = df.loc[i]
            in_t = datetime.datetime.fromtimestamp(obj['DateTimeInTick'] / 1000)
            out_t = datetime.datetime.fromtimestamp(obj['DateTimeOutTick'] / 1000)
            month_1 = in_t.month
            month_2 = out_t.month
            weekday_1 = in_t.weekday()
            weekday_2 = out_t.weekday()
            halfhour_1 = 2 * in_t.hour + 1 + int(in_t.minute >= 30)
            halfhour_2 = 2 * in_t.hour + 1 + int(in_t.minute >= 30)
            efficiency = obj['MacCountAll'] / obj['Duration']
            file.write(f"{month_1},{weekday_1},{halfhour_1},{round(efficiency, 2)},{obj['DateTimeOutTick']},{obj['MacCountAll']}\n")
            if month_1 != month_2 or weekday_1 != weekday_2 or halfhour_1 != halfhour_2:
                file.write(f"{month_2},{weekday_2},{halfhour_2},{round(efficiency, 2)},{obj['DateTimeOutTick']},{obj['MacCountAll']}\n")
        file.close()
        os.rename(f"{folder}/{filename}", f"{folder}/success_{filename}")
# parquet_to_csv(["./data/RowData/player_log/player=1547/month=2020-11.parquet"])
# th1 = Thread(target=parquet_to_csv, args=([
#     "./data/RowData/player_log/player=1547/month=2020-11.parquet",
# ], ))

# th2 = Thread(target=parquet_to_csv, args=([
#     "./data/RowData/player_log/player=1547/month=2020-12.parquet",
# ], ))
# # th3 = Thread(target=parquet_to_csv, args=([
# #     "./data/RowData/player_log/player=1547/month=2020-11.parquet",
# # ], ))

# # th4 = Thread(target=parquet_to_csv, args=([
# #     "./data/RowData/player_log/player=1547/month=2020-12.parquet",
# # ], ))
# th1.start()
# th2.start()

In [ ]:
parquet_to_csv([
    "./data/RowData/player_log/player=1547/month=2021-01.parquet",
    "./data/RowData/player_log/player=1547/month=2021-02.parquet"
])

In [50]:
import re
def getConstructionId(path):
    num = re.search(r'player=\d*', path)
    return int(num[0][7:])

getConstructionId('data/RowData/player_log/player=1547/success_month=2020-11.csv')

details = pd.read_csv('data/player_details.csv')
details

# with open('data/RowData/player_log/player=1547/success_month=2020-11.csv') as rd:
#     items = rd.read().split('\n')
# items
# for i, strr in enumerate(items):
#     if i == 1:
#        strr += ',distances' 
#     else:
#         strr += ','

,,PlayerId;PlayerNumber;mediaType;GeoLat;GeoLon;Face
257;NVS036APL;dbb;55,04145;82,913128;A
258;NVS053APL;dbb;55,032078;82,931467;A
259;NVS024BPL;dbb;55,035644;83,001675;B
260;NVS002APL;dbb;55,066428;82,929283;A
261;NVS037APL;dbb;55,043756;82,906317;A
262;NVS031APL;dbb;55,055328;82,901589;A
263;NVS054APL;dbb;55,041056;82,925789;A
264;NVS032APL;dbb;55,055925;82,903022;A
265;NVS058APL;dbb;54,999647;82,870419;A
266;NVS017APL;dbb;55,038117;82,978169;A


In [5]:
import pandas as pd
import pyarrow.parquet as pq
import datetime

In [6]:
df = pq.read_table(source="./data/RowData/player_log/player=1547/month=2020-11.parquet").to_pandas()

In [8]:
print(df.shape)
new_df = pd.DataFrame(columns=('month', 'week_day', 'half_hour', 'efficiency', 'unix', 'views'), )
new_df

(494703, 14)


,month,week_day,half_hour,efficiency,unix,views


In [6]:
parquet_to_csv([
    "./data/RowData/player_log/player=1547/month=2020-11.parquet",
    "./data/RowData/player_log/player=1547/month=2020-12.parquet"
])

  5%|▌         | 26645/494703 [00:11<03:21, 2324.04it/s]


KeyboardInterrupt: 

In [ ]:
new_df.shape

In [9]:
new_df

,month,week_day,half_hour,efficiency,unix,views


In [ ]:
new_df.to_csv('./preprocessing_data.csv')


In [24]:
import math
def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

center_coords = 55.030167631283575, 82.92179748091942
longest_coords = 54.80053466086988, 83.11027315692883
custom_coords = 54.96091109214436, 82.95228230422312
math.dist(center_coords, longest_coords)
#math.dist(center_coords, custom_coords)
#28, 26 км

0.29707639008884446